In [9]:
import s3fs
import boto3
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ec2-user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
BT_UL = pd.read_csv('LDA_train.csv')
BT_L = pd.read_csv('LDA_test.csv')

/tmp/ipykernel_30334/2975459504.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  BT_UL = pd.read_csv('LDA_train.csv')


In [4]:
BT_UL

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,bookId,title,series,author,rating,description,language,...,Fantasy,Erotica,History,Dystopia,Poetry,Biography,Manga,Thriller,Graphic Novels,Romance
0,1000,1000,28797,274618.The_Complete_Stories_of_Theodore_Sturge...,"The Complete Stories of Theodore Sturgeon, Vol...",The Complete Stories of Theodore Sturgeon #2,"Theodore Sturgeon, Paul Williams (Editor), Sam...",4.21,The second of a planned 10 volumes that will r...,English,...,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,1001,6612,14367051-city-of-bones-city-of-ashes-city-of-g...,City of Bones / City of Ashes / City of Glass ...,The Mortal Instruments #1-5,Cassandra Clare (Goodreads Author),4.58,The first five books in the #1 New York Times ...,English,...,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romance
2,1002,1002,18812,35156.Clash_of_the_Titans,Clash of the Titans,NaN,Alan Dean Foster,3.88,"PLAYTHING OF THE GODSHe was Perseus, son of Ze...",English,...,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,1003,4134,45791.The_Ballad_of_the_Sad_Caf_and_Other_Stories,The Ballad of the Sad Café and Other Stories,NaN,Carson McCullers,4.00,A classic work that has charmed generations of...,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,1004,4052,20603820-the-will,The Will,Magdalene #1,Kristen Ashley (Goodreads Author),4.29,"Early in her life, Josephine Malone learned th...",English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28647,29647,29647,38855,26162284-katy,Katy,NaN,Jacqueline Wilson,4.36,"Katy Carr is a lively, daredevil oldest sister...",English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28648,29648,29648,11399,29762106-the-anonymous-man,The Anonymous Man,The Anonymous Man #1,Vincent L. Scarsella (Goodreads Author),3.83,"What if one day you could become anonymous, fr...",English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28649,29649,29649,2239,608287.Jesus_Son,Jesus' Son,NaN,Denis Johnson,4.10,"Jesus' Son, the first collection of stories by...",English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thriller,NaN,NaN
28650,29650,29650,47747,293453.Akiko_in_the_Castle_of_Alia_Rellapor,Akiko in the Castle of Alia Rellapor,Akiko Books #4,Mark Crilley,4.06,"Akiko and her crew - Spuckler Boach, Mr. Beeba...",English,...,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Graphic Novels,NaN


In [5]:
def preprocess_text(text):

  #lowercase text
  text_preprocessed = text.lower()
  #remove punctuation
  text_preprocessed = re.sub(r'[^a-zA-Z ]+', '', text_preprocessed)
  #tokenize for stopword removal
  text_preprocessed = word_tokenize(text_preprocessed)
  #remove stopwords
  text_preprocessed = [word for word in text_preprocessed if word not in stopwords.words('english')]
  #join to make string again
  #text_preprocessed = (" ").join(text_preprocessed)

  return text_preprocessed

In [10]:
BT_UL['tokens'] = BT_UL['description'].apply(lambda x: preprocess_text(x))

In [12]:
BT_UL.to_csv('BERTopic_Unlabeled.csv')

In [13]:
BT_L['tokens'] = BT_L['description'].apply(lambda x: preprocess_text(x))
BT_L.to_csv('BERTopic_Labeled.csv')

In [14]:
BT_L

,Unnamed: 0.2,Unnamed: 0.1,index,Unnamed: 0,bookId,title,series,author,rating,description,...,relationships,break ups,divorce,weddings,death,family,friendship,labeled?,Contains True?,tokens
0,0,0,0,"39,822",34838660-not-part-of-the-plan,Not Part of the Plan,Blue Moon #4,Lucy Score (Goodreads Author),4.46,From the Wall Street Journal and #1 Amazon bes...,...,True,False,False,False,False,False,False,Yes,1.0,"[wall, street, journal, amazon, bestselling, a..."
1,1,1,1,"34,235",20176552-dragon-age-volume-1,"Dragon Age, Volume 1",Dragon Age Graphic Novels #1-3,"David Gaider, Chad Hardin (Illustrator), Antho...",4.26,Helping set the stage for BioWare's hotly anti...,...,False,False,False,False,False,False,False,Yes,0.0,"[helping, set, stage, biowares, hotly, anticip..."
2,2,2,2,"27,904",124110.Dangerous_to_Know,Dangerous to Know,NaN,Barbara Taylor Bradford (Goodreads Author),3.73,"Sebastian Locke, the fifty-six-year-old patria...",...,True,False,True,False,True,True,False,Yes,1.0,"[sebastian, locke, fiftysixyearold, patriarch,..."
3,3,3,3,"10,515",1046450.The_Wheel_of_Fortune,The Wheel of Fortune,NaN,Susan Howatch,4.11,"""Take me back to Oxmoon, the lost paradise of ...",...,True,False,False,False,False,True,False,Yes,1.0,"[take, back, oxmoon, lost, paradise, childhood..."
4,4,4,4,935,872333.Blue_Bloods,Blue Bloods,Blue Bloods #1,Melissa de la Cruz (Goodreads Author),3.69,"When the Mayflower set sail in 1620, it carrie...",...,False,False,False,False,False,False,False,Yes,0.0,"[mayflower, set, sail, carried, board, men, wo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,995,"17,361",588326.The_Blue_Helmet,The Blue Helmet,NaN,William Bell,3.42,Lee wants to be a Tarantula – a member of the ...,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1.0,"[lee, wants, tarantula, member, biggest, power..."
996,996,996,996,"9,029",93007.The_Merry_Adventures_of_Robin_Hood,The Merry Adventures of Robin Hood,NaN,Howard Pyle,4.07,The Merry Adventures of Robin Hood of Great Re...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[merry, adventures, robin, hood, great, renown..."
997,997,997,997,"32,216",1085376.Before_You_Sleep,Before You Sleep,NaN,"Linn Ullmann, Tiina Nunnally (Translator)",3.34,Moving from present-day Oslo to Brooklyn in th...,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1.0,"[moving, presentday, oslo, brooklyn, sleep, te..."
998,998,998,998,"1,036",28195.Inkspell,Inkspell,Inkworld #2,"Cornelia Funke (Goodreads Author), Anthea Bell...",3.91,"The captivating sequel to INKHEART, the critic...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[captivating, sequel, inkheart, critically, ac..."
